In [25]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Model

import numpy as np
import requests as rq
import io, h5py

In [26]:
data = rq.get('https://www.dropbox.com/s/c3umbo5y13sqcfp/synthetic_dataset.h5?raw=true')
data.raise_for_status()

with h5py.File(io.BytesIO(data.content), 'r') as dataset:
    x_train = np.array(dataset['X_train']).astype(np.float32).transpose([0, 2, 1])
    y_train = np.array(dataset['Y_train']).astype(np.float32)
    x_valid = np.array(dataset['X_valid']).astype(np.float32).transpose([0, 2, 1])
    y_valid = np.array(dataset['Y_valid']).astype(np.int32)
    x_test = np.array(dataset['X_test']).astype(np.float32).transpose([0, 2, 1])
    y_test = np.array(dataset['Y_test']).astype(np.int32)

print(x_train.shape, y_train.shape)

(21000, 200, 4) (21000, 12)


In [50]:
input_layer = layers.Input(shape=(200, 4))

conv1 = layers.Conv1D(filters=200, kernel_size=13, padding='same', name='conv1')(input_layer)
pool1 = layers.MaxPool1D(pool_size=6)(conv1)

positions = tf.range(pool1.shape[1])
context = layers.Embedding(input_dim=pool1.shape[1], output_dim=pool1.shape[2])(positions)

contextual_meaning = tf.add(pool1, context)

query = layers.Dense(100, name='query')(contextual_meaning)
value = layers.Dense(100)(contextual_meaning)

attention, weights = layers.MultiHeadAttention(num_heads=8, key_dim=32)(query, value, return_attention_scores=True)

fcl1 = layers.Dense(512, activation='relu')(attention)
fcl2 = layers.Dense(512, activation='relu')(fcl1)

flat = layers.Flatten()(fcl2)

output = layers.Dense(12, activation='sigmoid')(flat)

In [51]:
model = Model(inputs=input_layer, outputs=output)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', tf.keras.metrics.AUC(curve='ROC', name='auroc')])
model.fit(x_train, y_train, epochs=50, validation_data=(x_valid, y_valid))

Epoch 1/50
657/657 [==============================] - 7s 8ms/step - loss: 0.4265 - accuracy: 0.0923 - auroc: 0.5024 - val_loss: 0.4092 - val_accuracy: 0.1473 - val_auroc: 0.5625
Epoch 2/50
657/657 [==============================] - 5s 7ms/step - loss: 0.3854 - accuracy: 0.1630 - auroc: 0.6245 - val_loss: 0.3488 - val_accuracy: 0.2123 - val_auroc: 0.7061
Epoch 3/50
657/657 [==============================] - 5s 7ms/step - loss: 0.3404 - accuracy: 0.2323 - auroc: 0.7146 - val_loss: 0.3372 - val_accuracy: 0.2250 - val_auroc: 0.7238
Epoch 4/50
657/657 [==============================] - 5s 7ms/step - loss: 0.3339 - accuracy: 0.2514 - auroc: 0.7359 - val_loss: 0.3234 - val_accuracy: 0.2923 - val_auroc: 0.7585
Epoch 5/50
657/657 [==============================] - 5s 7ms/step - loss: 0.3112 - accuracy: 0.3177 - auroc: 0.7907 - val_loss: 0.2843 - val_accuracy: 0.3607 - val_auroc: 0.8446
Epoch 6/50
657/657 [==============================] - 5s 7ms/step - loss: 0.2626 - accuracy: 0.4006 - auroc: 0